# LIME

LIME is 'locally interpetable model-agnostic explanations'.

The paper is at http://arxiv.org/pdf/1602.04938v1.pdf , by Ribeiro, Singh, and Guestrin.  Ribeiro has a blog post about it at https://homes.cs.washington.edu/~marcotcr/blog/lime/ . There is code provided by Ribeiro at https://github.com/marcotcr/lime

There is thus already ample documentation and code about LIME, and this repo is for self-study purposes primarily, and likely wont introduce anything much new to the world, for now :-)

## What LIME does

LIME does the following:
- creates interpretable features, which for sparse nlp models means, bag of unigram words, containing all the words in the training vocabulary (I think)
- samples from interpretable feature space, near an example we wish to explain
- uses local gradients, from near the target example, to explain which interpretable features most affect decisions around that example

## LIME Experiments

### Train and test distributions differ

- train on `news20`, for atheist vs christianity
- test against new [religion](https://github.com/marcotcr/lime-experiments/blob/master/religion_dataset.tar.gz) dataset, created from websites from from [DMOZ](https://github.com/marcotcr/lime-experiments/blob/master/religion_dataset.tar.gz) directory
  - these data points have similar classes to the news20 training sets, ie atheism vs christianity.  However, the features are fairly different, and eg learning the names of prolific atheist posters in news20 wont generalize to the DMOZ websites.
- the idea is to examine to what extent the LIME explanations (or any other explanations for that matter) can facilitate rmeoving 'junk' features, after/during training, and thus improving the score on the DMOZ-derived dataset

## Downloading data, and training simple linear model

Let's start by downloading the datasets, and training a simple linear model:

### Download data

In [29]:
from sklearn.datasets import fetch_20newsgroups
# This is in a python script in same directory:
from religion_dataset import fetch_religion


global_categories = ['atheism', 'religion']
news_categories = ['alt.atheism', 'soc.religion.christian']

twenty_train = fetch_20newsgroups(subset='train', categories=news_categories, shuffle=True, random_state=123)
twenty_test = fetch_20newsgroups(subset='test', categories=news_categories, shuffle=True, random_state=123)
religion_test = fetch_religion()

loading religion dataset to memory...
class_id_by_name {'christianity': 1, 'atheism': 0}
failed to decode to utf-8 => skipping 1 doc
... religion dataset loaded


### Train a model

In [79]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
import numpy as np


class Model(object):
    def __init__(self, trainer):
        self.trainer = trainer
        trainers = {
            'nb': MultinomialNB(),
            'sgd': SGDClassifier(loss='hinge', penalty='l2',
                                 alpha=1e-3, n_iter=5, random_state=123),
            'rbf': SVC(C=1000000, kernel='rbf')
        }
        self.model = trainers[trainer]
        print('trainer: %s' % trainer)

    def train(self, bunch):
        self.count_vect = CountVectorizer()
        self.X_train_counts = self.count_vect.fit_transform(bunch.data)

        self.tfidf_transformer = TfidfTransformer()
        self.X_train_tfidf = self.tfidf_transformer.fit_transform(self.X_train_counts)

        self.model.fit(self.X_train_tfidf, bunch.target)
        train_pred = self.model.predict(self.X_train_tfidf)
        train_num_right = np.equal(train_pred, bunch.target).sum()
        return train_num_right / len(bunch.target) * 100

    def confidence_from_counts(self, counts):
        tfidf = self.tfidf_transformer.transform(counts)
        confidence = self.model.decision_function(tfidf)
        return confidence
    
    def test(self, bunch):
        X_test_counts = self.count_vect.transform(bunch.data)

        X_test_tfidf = self.tfidf_transformer.transform(X_test_counts)
        test_pred = self.model.predict(X_test_tfidf)
        test_num_right = np.equal(test_pred, bunch.target).sum()
        return test_num_right / len(bunch.target) * 100

model = Model('rbf')

print('twenty train:', model.train(twenty_train))
print('twenty test:', model.test(twenty_test))
print('religion test:', model.test(religion_test))

trainer: rbf
twenty train: 100.0
twenty test: 93.8633193863
religion test: 56.8376068376


We can see that the model gets almost perfect accuracy on `twenty-news` training data, fairly good accuracy on `twenty-news` test data, and, as per the LIME paper, less good accuracy on the `religion` test data.  The model doesnt generalize well.

## LIME Model

### LIME Generalized Model

LIME trains a model $\xi$, drawn from a class $G$ of interpretable models.  Where interpretable models for LIME means simple-ish linear models, such as linear models, decision trees, or falling rule lists.  $\xi$ is the solution to:

$$\xi(x) = \mathrm{argmin}_{g \in G} \left( \mathcal{L}(f, g,\Pi_x)+\Omega(g) \right)$$

Where:
- $G$ is class of interpretable models
- $f$ is the function learned by the network we wish to interpret
- $\Pi_x(z)$ is a measure of proximity of $z$ to $x$
- $\mathcal{L}$ is a measure of how unfaithful $g$ is in representing $f$ in the locality defined by $\Pi(x)$
- $\Omega(\cdot)$ is a measure of complexity

### LIME Specialized Model

This is a general formulation. For LIME, we add additional constraints and assumptions
- $G$ is taken to be the class of linear models, and in particular: $g(z') = w_g \cdot z'$, where $w_g$ are parameters to be learned
- $\Pi(x)$ is defined as: $\exp \left( \frac{ -D(x, z)^2 } {\sigma^2} \right)$, so it's something like a radial basis function, and is close to one near $x$, then falls off with distance
- $\mathcal{L}$ is the square loss, weighted by locality:

$$\mathcal{L}(f, g, \Pi_x) = \sum_{z, z', \mathcal{Z}} \Pi_x(z) \left( f(z) - g(z') \right)^2 $$

## Interpretable features for NLP

The locally interpretable features are binary, $\mathbf{x}' \in \{0,1\}^{d'}$. ~~for nlp, LIME uses LARS to obtain the $K$ most important features/words from the model.  I think.  I think these interpretable features are global.  Again, I'm not entirely 100% sure on this point currently :-)~~

For nlp, I think that the interpretable features are a bag of unigrams.  The unigrams includes all the entire vocabulary, I think.  Samples are drawn from this (presumably by perturbing the original example-to-be-explained slightly), then LARS path is run against these samples, to obtain the top $K$ explainers. I think.

## Drawing samples

So, first we should draw samples.  I'm doubling the number of perturbations to each sample until the similarity drops below 0.1.  Its not ideal since the number of perturbations should probably be more like a probability, or percentage, than an absolute number, otherwise it will not handle varying document lengths very well.  But it's a start.

Once we have the number of perturbations. We sample a number of perturbations up to that, for each document, then replace this number of features in the example with some random features.

We should probably draw samples of total number of features in each sample, rather than keeping this fixed, and just swapping the features around.  For now though, swapping features around seems like a good start.

Using classification as the result of $f$ didnt work very well for me: all the perturbed samples systematically had the same class as the original class.  However, by using the confidence score, instead of the prediced class, the samples seem to have reasonable diversity of confidence scores, albeit without sufficient diversity to change class.

In [80]:
from sklearn import linear_model
from sklearn.metrics.pairwise import cosine_distances
from scipy.sparse import csr_matrix
import random

N = 15000   # number of samples, from section 5.1 of the paper
N = 5000
rho = 25   # from https://github.com/marcotcr/lime-experiments/blob/master/generate_data_for_compare_classifiers.py#L62
# distance is calculated as per https://github.com/marcotcr/lime-experiments/blob/master/explainers.py#L115:
"""
distance_fn = lambda x : sklearn.metrics.pairwise.cosine_distances(x[0],x)[0] * 100
"""

def sample_around(x, X, y, N, f, similarity_kernel, ifspace_to_inputspace_fn):
    """
    For doing the perturbations, we will double the number of perturbations each sample, until
    the effect of the distance_fn
    """
    example_list = list(x.nonzero()[1])
    active_set = set(x.nonzero()[1])
    n = 0
    samples = []
    num_perturbations = 1
    num_features = X.shape[1]
    active_set_size = len(active_set)
    z_example = csr_matrix((
        [1] * active_set_size,
        ([0] * active_set_size, example_list)
    ), shape=(1, num_features))
    calibrating_perturbations = True
    while n < N:
        if calibrating_perturbations:
            print('n', n, 'calib', calibrating_perturbations, 'num_perturbations', num_perturbations)
            new_list = list(active_set)
            active_set_idxs = np.random.choice(active_set_size, size=(num_perturbations,))
            swap_idxs = np.random.choice(num_features, size=(num_perturbations,))
            for i, active_set_idx in enumerate(active_set_idxs):
                new_list[active_set_idx] = swap_idxs[i]
            z_ifspace = csr_matrix((
                [1] * active_set_size,
                ([0] * active_set_size, new_list)
            ), shape=(1, num_features))
            similarity = similarity_kernel(z_example, z_ifspace)
            if similarity > 0.1:
                num_perturbations *= 2
                if num_perturbations > active_set_size:
                    num_perturbations = active_set_size
                    calibrating_perturbations = False
            else:
                calibrating_perturbations = False
        else:
            this_num_perturbations = random.randint(1, num_perturbations)
            new_list = list(active_set)
            active_set_idxs = np.random.choice(active_set_size, size=(this_num_perturbations,))
            swap_idxs = np.random.choice(num_features, size=(this_num_perturbations,))
            for i, active_set_idx in enumerate(active_set_idxs):
                new_list[active_set_idx] = swap_idxs[i]
            z_ifspace = csr_matrix((
                [1] * active_set_size,
                ([0] * active_set_size, new_list)
            ), shape=(1, num_features))
            similarity = similarity_kernel(z_example, z_ifspace)
        z_inputspace = ifspace_to_inputspace_fn(z_ifspace)
        pred = f(z_inputspace)[0]
        sample = {
            'similarity': similarity,
            'z_ifspace': z_ifspace,
            'z_inputspace': z_inputspace,
            'pred': pred
        }
        samples.append(sample)
        if (n + 1) % 1000 == 0:
            print(n + 1)
        n += 1
    return samples

def my_cosine_distance(v1, v2):
    """
    As per https://github.com/marcotcr/lime-experiments/blob/master/explainers.py#L115
    """
    return cosine_distances(v1, v2)[0][0] * 100

def create_rbf_similarity_kernel(sigma, distance_fn):
    sigma_squared = sigma * sigma
    def my_similarity_kernel(v1, v2):
        d = distance_fn(v1, v2)
        return np.exp(-d * d / sigma_squared)
    return my_similarity_kernel

def tfidf_ifspace_to_inputspace_fn(z_ifspace):
    z_inputspace = model.tfidf_transformer.transform(z_ifspace)
    return z_inputspace

my_similarity_kernel = create_rbf_similarity_kernel(sigma=rho, distance_fn=my_cosine_distance)
samples = sample_around(
    x=model.X_train_tfidf[2], X=model.X_train_tfidf, y=twenty_train.target, N=N,
    f=model.confidence_from_counts,
    similarity_kernel=my_similarity_kernel,
    ifspace_to_inputspace_fn=tfidf_ifspace_to_inputspace_fn)
print('sampling done')

n 0 calib True num_perturbations 1
n 1 calib True num_perturbations 2
n 2 calib True num_perturbations 4
n 3 calib True num_perturbations 8
n 4 calib True num_perturbations 16
n 5 calib True num_perturbations 32
n 6 calib True num_perturbations 64
1000
2000
3000
4000
5000
sampling done


## Weighted Lasso Lars

For LIME, we need to run through Lasso LARS path, for the loss function stated above, ie:

$$\mathcal{L}(f, g, \Pi_x) = \sum_{z, z', \mathcal{Z}} \Pi_x(z) \left( f(z) - g(z') \right)^2 $$

The proximity function is weighting the samples in the loss function.  Otherwise this is standard rms-loss linear regression.  Looking at sklearn library, [LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) allows one to provide `sample_weight`s, however, [LassoLars](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLars.html#sklearn.linear_model.LassoLars), which is what we want to use, does not.  We could implement Lasso Lars by hand, but I imagine there are lots of little tweaks and tricks inside the sklearn implementation, so I'm just going to wrap the sklearn one.

Let's examine a general weighted mse-loss linear regression function:

$$\mathcal{L} = \sum_n (y_n - f(x_n))^2 $$

If we have per-sample weights, $\phi_n$, then this becomes:

$$\mathcal{L} = \sum_n \phi_n(y_n - f(x_n))^2 $$

We can move the $\phi_n$ inside:

$$\mathcal{L} = \sum_n ( y_n\sqrt{\phi_n} - f(x_n)\sqrt{\phi_n})^2 $$

... and note that $f(x_n)$ is $\mathbf{w} \cdot \mathbf{x}_n$, for parameter vector $\mathbf{w}$, to be learned.  So we can write:

$$\mathcal{L} = \sum_n (y_n \sqrt{\phi_n} - \sum_j( w_j (x_{n,j}\sqrt{\phi_n})))^2 $$

So, we simply need to scale the $\mathbf{x}_n$ and $y_n$ values appropriately, by $\sqrt{\phi_n}$.  The only thing to be a bit careful of is that the analysis above doesnt include a bias/intercept.  However, by default both `LinearRegression` and `LassoLars` regression do, in the sklearn implementation, and it's so by default.  So, we'll need to handle the intercept ourselves, by adding an additional all-1s feature, and turn off the intercept in `LassoLars`.

Writing as a class this gives:

In [71]:
from scipy.sparse import lil_matrix
import math


class WeightedLassoLars(object):
    def __init__(self, **kwargs):
        self.fit_intercept = kwargs.get('fit_intercept', True)
        kwargs['fit_intercept'] = False
        self.lasso = linear_model.LassoLars(**kwargs)

    @staticmethod
    def add_intercept_feature(X):
        # add intercept feature, all 1s
        N = X.shape[0]
        K = X.shape[1]
#         X_new = np.zeros((N, K + 1), dtype=np.float32)
        X_new = lil_matrix((N, K + 1), dtype=np.float32)
        X_new[:, 1:] = X
        X_new[:, 0] = 1
        return X_new
        
    def fit(self, X, y, sample_weight=None):
        if self.fit_intercept:
            X = self.add_intercept_feature(X)
        if sample_weight is not None:
            for n, weight in enumerate(sample_weight):
                X[n] *= math.sqrt(weight)
                y[n] *= math.sqrt(weight)
        self.lasso.fit(X.toarray(), y)
    
    def predict(self, X):
        if self.fit_intercept:
            X = self.add_intercept_feature(X)
        return self.lasso.predict(X.toarray())
    
    @property
    def active_(self):
        res = []
        for j in self.lasso.active_:
            if j > 0:
                res.append(j - 1)
        return res

## Solving the Lasso Lars for our samples

So, now we just have to plug the sampled data into the new WeightedLassoLars class.  In theory.

In [81]:
from scipy.sparse import lil_matrix


K = 10  # I *think* the paper uses K as 10

lasso = WeightedLassoLars(alpha=0, max_iter=K + 1)  # +1, because intercept gets chosen first..
samples_N = len(samples)
ifspace_num_features = samples[0]['z_ifspace'].shape[1]
samples_X_ifspace = lil_matrix((samples_N, ifspace_num_features), dtype=np.float32)
samples_y = np.zeros((samples_N,), dtype=np.float32)
samples_weights = np.zeros((samples_N,), dtype=np.float32)
for n, sample in enumerate(samples):
    samples_X_ifspace[n] = sample['z_ifspace']
    samples_y[n] = sample['pred']
    samples_weights[n] = sample['similarity']
lasso.fit(samples_X_ifspace, samples_y, samples_weights)
print('lasso fitted')

lasso fitted


print the active features:

In [82]:
print(lasso.active_)
for j in lasso.active_:
    print(model.count_vect.get_feature_names()[j])

[11546, 11430, 12546, 10640, 10155, 15469, 3410, 4478, 6739, 11706]
mathew
mantis
nntp
kmr4
isc
rit
bobby
co
eg
men
